In [7]:
import dash
from dash import dcc, html, dash_table, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import timedelta

# =====================================================================
# 1. DADOS
# =====================================================================
print("--- A CARREGAR DADOS... ---")

grupos_analise = {
    'Varejo (Risco)': ['MGLU3.SA', 'LREN3.SA', 'BHIA3.SA', 'AMER3.SA'],
    'Tech & Pagamentos': ['PAGS34.SA', 'TOTS3.SA'], 
    'Bancos Tradicionais': ['ITUB4.SA', 'BBDC4.SA', 'BBAS3.SA', 'SANB11.SA'],
    'Bancos Digitais': ['ROXO34.SA', 'INBR32.SA', 'BPAN4.SA'],
    'Commodities': ['VALE3.SA', 'PETR4.SA', 'CMIN3.SA'],
    'Construtoras': ['CYRE3.SA', 'EZTC3.SA', 'MRVE3.SA'],
    'Macro (Dólar/Ibov)': ['^BVSP', 'USDBRL=X']
}

mapa_nomes = {
    'MGLU3.SA': 'Magalu', 'LREN3.SA': 'Lojas Renner', 'BHIA3.SA': 'Casas Bahia', 'AMER3.SA': 'Americanas',
    'PAGS34.SA': 'PagBank', 'TOTS3.SA': 'Totvs',
    'ITUB4.SA': 'Itaú Unibanco', 'BBDC4.SA': 'Bradesco', 'BBAS3.SA': 'Banco do Brasil', 'SANB11.SA': 'Santander',
    'ROXO34.SA': 'Nubank', 'INBR32.SA': 'Banco Inter', 'BPAN4.SA': 'Banco Pan',
    'VALE3.SA': 'Vale', 'PETR4.SA': 'Petrobras', 'CMIN3.SA': 'CSN Mineração',
    'CYRE3.SA': 'Cyrela', 'EZTC3.SA': 'Eztec', 'MRVE3.SA': 'MRV Engenharia',
    '^BVSP': 'Ibovespa', 'USDBRL=X': 'Dólar'
}

todos_tickers = [item for sublist in grupos_analise.values() for item in sublist]

# Ajuste: Adicionei auto_adjust=False para evitar warnings futuros, mas mantive a lógica
dados = yf.download(todos_tickers, start='2022-01-01', auto_adjust=True, ignore_tz=True)

if 'Close' in dados.columns: 
    df_close = dados['Close']
else: 
    df_close = dados

df_close = df_close.ffill()
print("--- DADOS PRONTOS! ---")

# =====================================================================
# 2. LAYOUT
# =====================================================================
app = dash.Dash(__name__)

estilo_lado_a_lado = {'display': 'flex', 'flexDirection': 'row', 'justifyContent': 'space-between', 'marginBottom': '20px'}
estilo_card = {'width': '49%', 'backgroundColor': 'white', 'padding': '15px', 'borderRadius': '10px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'}

app.layout = html.Div(style={'backgroundColor': '#ecf0f1', 'padding': '25px', 'fontFamily': 'Segoe UI, sans-serif'}, children=[
    
    html.H1("Analytics Financeiro & Simulação Estocástica", style={'textAlign': 'center', 'color': '#2c3e50'}),
    
    # Menu Superior
    html.Div([
        html.Div([html.Label("Grupo A:", style={'fontWeight': 'bold', 'color': '#2980b9'}),
                  dcc.Dropdown(id='drop-1', options=[{'label': k, 'value': k} for k in grupos_analise.keys()], value=['Varejo (Risco)'], multi=True)], style={'width': '48%'}),
        html.Div([html.Label("Grupo B:", style={'fontWeight': 'bold', 'color': '#c0392b'}),
                  dcc.Dropdown(id='drop-2', options=[{'label': k, 'value': k} for k in grupos_analise.keys()], value=['Bancos Tradicionais'], multi=True)], style={'width': '48%'})
    ], style=estilo_lado_a_lado),

    # Gráficos de Linha
    html.Div([html.Div([dcc.Graph(id='grafico-linha-1')], style=estilo_card), html.Div([dcc.Graph(id='grafico-linha-2')], style=estilo_card)], style=estilo_lado_a_lado),
    
    # Gráficos Risco/Retorno e Boxplot
    html.Div([html.Div([dcc.Graph(id='grafico-risco-retorno', style={'height': '350px'})], style=estilo_card), html.Div([dcc.Graph(id='grafico-boxplot', style={'height': '350px'})], style=estilo_card)], style=estilo_lado_a_lado),

    # --- SIMULAÇÃO DE MONTE CARLO ---
    html.H2("Laboratório Preditivo (Intervalo de Confiança)", style={'textAlign': 'center', 'marginTop': '40px', 'color': '#8e44ad'}),
    
    html.Div([
        html.Div([
            html.H4("Parâmetros", style={'marginTop': '0'}),
            html.Label("Ativo:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(id='drop-simulacao', options=[{'label': mapa_nomes.get(t, t), 'value': t} for t in todos_tickers], value='ROXO34.SA', clearable=False),
            html.Br(),
            html.Label("Projeção Temporal:", style={'fontWeight': 'bold'}),
            dcc.Slider(id='slider-dias', min=30, max=360, step=30, value=90, marks={30:'1 Mês', 180:'6 Meses', 360:'1 Ano'}),
            html.Br(),
            html.P("O gráfico agora conecta os dados históricos recentes (preto) com a projeção futura (azul/vermelho) usando datas reais.", style={'fontSize': '12px', 'color': 'gray', 'textAlign': 'justify'})
        ], style={'width': '20%', 'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 'marginRight': '20px', 'height': '400px'}),
        
        html.Div([dcc.Graph(id='grafico-monte-carlo', style={'height': '440px'})], style={'width': '78%', 'backgroundColor': 'white', 'padding': '10px', 'borderRadius': '10px'})
    ], style={'display': 'flex', 'flexDirection': 'row', 'marginBottom': '40px'}),

    html.H3("Relatório Quantitativo", style={'textAlign': 'center'}),
    html.Div([html.Div(id='tabela-kpi')], style={'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '10px', 'marginBottom': '50px'}),

    html.Hr(), # Linha separadora final

    # --- RODAPÉ COM DISCLAIMER (ADICIONADO) ---
    html.Div([
        html.P("⚠️ Disclaimer: Este dashboard foi desenvolvido para fins estritamente acadêmicos e educacionais. "
               "Os dados apresentados não constituem recomendação de compra ou venda de ativos. "
               "As simulações de Monte Carlo são modelos matemáticos probabilísticos e não garantem retornos futuros."),
        
        html.P(f"Desenvolvido por Igor Carvalho • Dados fornecidos pelo Yahoo Finance.", style={'marginTop': '10px'}),
        
        html.P("🔧 Nota Técnica: Caso encontre erro de conexão (Address already in use), altere a variável 'port' no final do código.", 
               style={'fontSize': '10px', 'color': '#999', 'marginTop': '5px'})

    ], style={'textAlign': 'center', 'color': 'gray', 'fontSize': '12px', 'padding': '20px', 'backgroundColor': '#f9f9f9'})
])

# =====================================================================
# 3. LÓGICA (CALLBACKS)
# =====================================================================
@app.callback(
    [Output('grafico-linha-1', 'figure'), Output('grafico-linha-2', 'figure'), 
     Output('grafico-risco-retorno', 'figure'), Output('grafico-boxplot', 'figure'), 
     Output('tabela-kpi', 'children'), Output('grafico-monte-carlo', 'figure')],
    [Input('drop-1', 'value'), Input('drop-2', 'value'), 
     Input('drop-simulacao', 'value'), Input('slider-dias', 'value')])
def atualizar_dashboard(lista1, lista2, ativo_simulacao, dias_projecao):
    
    # Tratamentos básicos
    if not lista1: lista1 = []
    if not lista2: lista2 = []
    tickers_grupo1 = [t for g in lista1 for t in grupos_analise.get(g, [])]
    tickers_grupo2 = [t for g in lista2 for t in grupos_analise.get(g, [])]
    todos_selecionados = list(set(tickers_grupo1 + tickers_grupo2))
    if not todos_selecionados: todos_selecionados = [ativo_simulacao]
    validos = [t for t in todos_selecionados if t in df_close.columns]
    
    df_main = df_close[validos].copy().dropna().rename(columns=mapa_nomes)
    retornos_diarios = df_main.pct_change().dropna()

    # Gráficos Padrão
    def criar_linha(tickers, t):
        n = [mapa_nomes.get(k, k) for k in tickers if k in df_close.columns]
        if not n: return px.line(title="Vazio")
        d = df_main[n]
        return px.line((d/d.bfill().iloc[0])*100, title=t, template='plotly_white')
    
    fig1 = criar_linha(tickers_grupo1, "Grupo A (Base 100)")
    fig2 = criar_linha(tickers_grupo2, "Grupo B (Base 100)")
    
    vol = retornos_diarios.std() * (252**0.5)
    ret_tot = (df_main.iloc[-1] / df_main.iloc[0]) - 1
    fig_rr = px.scatter(x=vol, y=ret_tot, color=df_main.columns, title="Risco vs Retorno", template='plotly_white')
    
    df_box = retornos_diarios.melt(var_name='Ativo', value_name='Retorno')
    fig_box = px.box(df_box, x='Ativo', y='Retorno', color='Ativo', title="Boxplot", template='plotly_white', points="outliers")
    fig_box.update_layout(showlegend=False)

    # --- SIMULAÇÃO DE MONTE CARLO ---
    if ativo_simulacao in df_close.columns:
        serie_preco = df_close[ativo_simulacao].dropna()
        preco_atual = serie_preco.iloc[-1]
        ultima_data = serie_preco.index[-1]
        
        # Parâmetros
        log_returns = np.log(1 + serie_preco.pct_change())
        u = log_returns.mean()
        var = log_returns.var()
        drift = u - (0.5 * var)
        stdev = log_returns.std()
        
        t_intervals = int(dias_projecao)
        iterations = 1000 
        
        # Matriz
        daily_returns = np.exp(drift + stdev * np.random.normal(0, 1, (t_intervals, iterations)))
        
        price_list = np.zeros_like(daily_returns)
        price_list[0] = preco_atual
        for t in range(1, t_intervals):
            price_list[t] = price_list[t - 1] * daily_returns[t]
            
        df_sim = pd.DataFrame(price_list)
        sim_p95 = df_sim.quantile(0.95, axis=1)
        sim_mean = df_sim.mean(axis=1)
        sim_p05 = df_sim.quantile(0.05, axis=1)
        
        nome_ativo = mapa_nomes.get(ativo_simulacao, ativo_simulacao)

        datas_futuras = pd.date_range(start=ultima_data, periods=t_intervals, freq='B')
        historico_recente = serie_preco.tail(60)
        
        fig_mc = go.Figure()

        fig_mc.add_trace(go.Scatter(x=historico_recente.index, y=historico_recente.values, mode='lines', line=dict(color='black', width=2), name='Histórico'))
        fig_mc.add_trace(go.Scatter(x=datas_futuras, y=sim_p95, mode='lines', line=dict(width=0), showlegend=False, name='Topo'))
        fig_mc.add_trace(go.Scatter(x=datas_futuras, y=sim_p05, mode='lines', line=dict(width=0), fill='tonexty', fillcolor='rgba(0, 100, 255, 0.2)', name='IC 95%'))
        fig_mc.add_trace(go.Scatter(x=datas_futuras, y=sim_mean, mode='lines', line=dict(color='royalblue', width=3, dash='dash'), name='Média'))
        
        ultima_data_proj = datas_futuras[-1]
        try:
            fig_mc.add_annotation(x=ultima_data_proj, y=sim_p95.iloc[-1], text=f"R${sim_p95.iloc[-1]:.2f}", showarrow=False, yshift=10, font=dict(color='green', size=11))
            fig_mc.add_annotation(x=ultima_data_proj, y=sim_p05.iloc[-1], text=f"R${sim_p05.iloc[-1]:.2f}", showarrow=False, yshift=-10, font=dict(color='red', size=11))
        except:
            pass

        fig_mc.update_layout(title=f"Projeção: {nome_ativo}", template='plotly_white', yaxis_title='Preço (R$)', hovermode="x unified")
    else:
        fig_mc = px.line(title="Ativo não encontrado")

    # Tabela KPI
    kpi_data = []
    for ativo in df_main.columns:
        r = retornos_diarios[ativo]
        v = r.std() * (252**0.5)
        kpi_data.append({'Ativo': ativo, 'Retorno': f"{((1+r).prod()-1)*100:.1f}%", 'Volatilidade': f"{v*100:.1f}%", 'VaR 95%': f"{r.quantile(0.05)*100:.2f}%"})
    df_kpi = pd.DataFrame(kpi_data)
    tabela = dash_table.DataTable(data=df_kpi.to_dict('records'), columns=[{'name': i, 'id': i} for i in df_kpi.columns], style_cell={'textAlign': 'center', 'fontFamily': 'Segoe UI', 'padding': '10px'}, style_header={'backgroundColor': '#2c3e50', 'color': 'white'})

    return fig1, fig2, fig_rr, fig_box, tabela, fig_mc

if __name__ == '__main__':
    # --- CONFIGURAÇÃO DA PORTA E AVISO NO TERMINAL ---
    minha_porta = 8091
    
    print(f"🚀 Iniciando Dashboard...")
    print(f"👉 Acesse em: http://127.0.0.1:{minha_porta}/")
    print(f"⚠️ Se der erro 'Address already in use', mude 'minha_porta' para {minha_porta+1}.")
    
    app.run(debug=True, port=minha_porta)

--- A CARREGAR DADOS... ---


[*********************100%***********************]  21 of 21 completed


--- DADOS PRONTOS! ---
🚀 Iniciando Dashboard...
👉 Acesse em: http://127.0.0.1:8067/
⚠️ Se der erro 'Address already in use', mude 'minha_porta' para 8068.
